# FP-growth高效发现频繁项集

对于Apriori算法，虽然已经提升了发现频繁项集的速度，但是当我们改变频繁项集大小时，依然需要重新遍历整个数据集来重新查找频繁项集，而FP-growth算法只需要对数据进行两次遍历，理论上速度可以提升两个数量级；

FP-growth算法基本流程：
1. 构建FP树；
2. 从FP树中挖掘频繁项集；

In [1]:
from numpy import *

import matplotlib.pyplot as plt

%matplotlib inline

dataPath = '../../../git_mlaction/machinelearninginaction/Ch12/'

## FP树：用于编码数据集的有效方式

典型的利用一种更合适的数据结构来结构化数据，从而使得后续对数据的操作变得更加简单、快速；

* 优点：一般情况下都要快于Apriori，尤其是大数据下，普遍要快两个数量级；
* 缺点：实现困难，在某些数据集上性能会下降；
* 适用数据类型：标称型；

类比：MySQL数据库等也用树结构来构建索引体系，加快索引速度，同样也是一个利用特殊的数据结构来实现业务上某方面的优化；

FP树结构理解，如下图：

![FP-growth](image/fp-growth.png)

构建过程由两步（两次对数据集的遍历）组成：
1. Apriori算法，查找统计所有元素出现此处保存到一个元素头指针字典中；
2. 使用频繁项集构建FP树，排序后遍历每个项集；
    1. 第一个元素是否已经出现在了树的第一层；
    2. 如果出现，则在对应节点上+1，同时判断第二个元素是否出现在了该节点的子节点，出现+1，没出现新建；
    3. 如果没出现，新建一个第一层节点；
    4. 新建任何节点都要注意添加link的问题（link是链接单个相同元素，因此link链的长度等于物品类别数）

树的节点保存的数据格式：{x:n}，x为物品种类，n为出现次数，树的路径表示项集中元素的数据，相似的节点（相似指同一物品种类）之间通过link链接起来；

## 构建FP树

### 创建FP树的数据结构

由于FP树结构较为复杂，包含节点、路径、link，节点又由两部分组成{类型:出现次数}，因此使用类而不是dict来描述该结构；

In [4]:
class FPTreeNode(object):
    def __init__(self, nameValue, numOccur, parentNode):
        '''
        nameValue:类别名
        numOccur:出现次数
        parentNode:父节点
        '''
        self.name = nameValue
        self.count = numOccur
        self.nodeLink = None
        self.parent = parentNode
        self.children = {}
        
    def inc(self, numOccur):
        '''
        该路径下元素出现次数
        '''
        self.count += numOccur
        
    def disp(self, ind=1):
        '''
        打印方法
        '''
        print '\t'*ind, self.name, ' : ', self.count
        for child in self.children.values():
            child.disp(ind+1)
            
dipped_node = FPTreeNode('dipped', 10, None)
milk_node = FPTreeNode('milk', 5, dipped_node)
beer_node = FPTreeNode('beer', 8, dipped_node)
dipped_node.children['milk'] = milk_node
dipped_node.children['beer'] = beer_node
beer_node.children['gum'] = FPTreeNode('gum', 2, beer_node)

dipped_node.disp()

	dipped  :  10
		beer  :  8
			gum  :  2
		milk  :  5


### 构建FP树

1. 遍历数据，获取每个元素项的出现频率，记录到头指针表中；
2. 使用头指针表对每个事务（即原始的各个组合）进行过滤、排序，得到新事务；
3. 使用新事务构建FP树；

In [50]:
# 创建FP树
def createFPTree(dataSet, minSup=1): # 注意此处minSup是出现次数的意思，不再是小数
    print dataSet
    # 第一次遍历数据，统计出现频率，存入头指针字典中
    # 统计
    headerTable = {}
    for trans in dataSet:
        for item in trans:
            headerTable[item] = headerTable.get(item, 0) + dataSet[trans] #???为啥是加这个货
    print headerTable
    # 过滤
    for k in headerTable.keys():
        if headerTable[k] < minSup:
            del headerTable[k]
    print headerTable
    freqItemSet = set(headerTable.keys()) # 频繁元素集
    if len(freqItemSet) <= 0:
        return None, None # 没有任何元素满足minSup需求
    for k in headerTable.keys():
        headerTable[k] = [headerTable[k], None]
    retTree = FPTreeNode('NULL', 1, None) # 上帝节点
    for tranSet, count in dataSet.items():
        localD = {}
        print 'start---'
        for item in tranSet:
            if item in freqItemSet:
                localD[item] = headerTable[item][0]
        print '------', localD
        if len(localD) > 0:
            orderedItems = [v[0] for v in sorted(localD.items(), key=lambda p: p[1], reverse=True)] # 对频繁项集进行排序
            print 'mid---', orderedItems
            print '------', headerTable
            updateFPTree(orderedItems, retTree, headerTable, count)
            retTree.disp()
        print 'end---'
    return retTree, headerTable

In [55]:
# 使用排序后的频繁项集更新FP树
def updateFPTree(items, inTree, headerTable, count):
    print items[0]
    # 更新树
    print 'step 1'
    if items[0] in inTree.children:
        print 'a'
        inTree.children[items[0]].inc(count)
    else:
        print 'b'
        inTree.children[items[0]] = FPTreeNode(items[0], count, inTree)
    inTree.disp()
    # 更新头指针表
    print 'step 2'
    if headerTable[items[0]][1] == None:
        print 'c'
        headerTable[items[0]][1] = inTree.children[items[0]]
    else:
        print 'd'
        print headerTable[items[0]][1]
        print (headerTable[items[0]][1]).name
        print (headerTable[items[0]][1]).nodeLink
        print inTree.children[items[0]]
        print (inTree.children[items[0]]).name
        updateHeader(headerTable[items[0]][1], inTree.children[items[0]])
    print 'step 3'
    # 递归处理剩余itmes中的元素项
    if len(items) > 1:
        updateFPTree(items[1::], inTree.children[items[0]], headerTable, count)

In [32]:
# 更新头指针表的指向
def updateHeader(elementNode, targetNode):
    while elementNode.nodeLink != None:
        elementNode = elementNode.nodeLink
    elementNode.nodeLink = targetNode

In [9]:
def loadSimpData():
    return [
        list('rzhjp'),
        list('zyxwvuts'),
        list('z'),
        list('rxnos'),
        list('yrxzqtp'),
        list('yzxeqstm')]
print loadSimpData()

[['r', 'z', 'h', 'j', 'p'], ['z', 'y', 'x', 'w', 'v', 'u', 't', 's'], ['z'], ['r', 'x', 'n', 'o', 's'], ['y', 'r', 'x', 'z', 'q', 't', 'p'], ['y', 'z', 'x', 'e', 'q', 's', 't', 'm']]


In [10]:
def createInitSet(dataSet):
    retDict = {}
    for trans in dataSet:
        retDict[frozenset(trans)] = 1
    return retDict

In [11]:
simpData = loadSimpData()
simpData

[['r', 'z', 'h', 'j', 'p'],
 ['z', 'y', 'x', 'w', 'v', 'u', 't', 's'],
 ['z'],
 ['r', 'x', 'n', 'o', 's'],
 ['y', 'r', 'x', 'z', 'q', 't', 'p'],
 ['y', 'z', 'x', 'e', 'q', 's', 't', 'm']]

In [12]:
initSet = createInitSet(simpData)
initSet

{frozenset({'e', 'm', 'q', 's', 't', 'x', 'y', 'z'}): 1,
 frozenset({'n', 'o', 'r', 's', 'x'}): 1,
 frozenset({'z'}): 1,
 frozenset({'s', 't', 'u', 'v', 'w', 'x', 'y', 'z'}): 1,
 frozenset({'p', 'q', 'r', 't', 'x', 'y', 'z'}): 1,
 frozenset({'h', 'j', 'p', 'r', 'z'}): 1}

In [56]:
simpTree, simpHeaderTable = createFPTree(initSet, 3)
simpTree.disp()

{frozenset(['e', 'm', 'q', 's', 't', 'y', 'x', 'z']): 1, frozenset(['x', 's', 'r', 'o', 'n']): 1, frozenset(['s', 'u', 't', 'w', 'v', 'y', 'x', 'z']): 1, frozenset(['q', 'p', 'r', 't', 'y', 'x', 'z']): 1, frozenset(['h', 'r', 'z', 'p', 'j']): 1, frozenset(['z']): 1}
{'e': 1, 'h': 1, 'j': 1, 'm': 1, 'o': 1, 'n': 1, 'q': 2, 'p': 2, 's': 3, 'r': 3, 'u': 1, 't': 3, 'w': 1, 'v': 1, 'y': 3, 'x': 4, 'z': 5}
{'s': 3, 'r': 3, 't': 3, 'y': 3, 'x': 4, 'z': 5}
start---
------ {'y': 3, 'x': 4, 's': 3, 'z': 5, 't': 3}
mid--- ['z', 'x', 'y', 's', 't']
------ {'s': [3, None], 'r': [3, None], 't': [3, None], 'y': [3, None], 'x': [4, None], 'z': [5, None]}
z
step 1
b
	NULL  :  1
		z  :  1
step 2
c
step 3
x
step 1
b
	z  :  1
		x  :  1
step 2
c
step 3
y
step 1
b
	x  :  1
		y  :  1
step 2
c
step 3
s
step 1
b
	y  :  1
		s  :  1
step 2
c
step 3
t
step 1
b
	s  :  1
		t  :  1
step 2
c
step 3
	NULL  :  1
		z  :  1
			x  :  1
				y  :  1
					s  :  1
						t  :  1
end---
start---
------ {'x': 4, 's': 3, 'r': 3}


KeyboardInterrupt: 